In [0]:
%load_ext autoreload
%autoreload 2


**Assumptions**
- Assuming product_id is the unique business key in products file, if any duplicate will pick up first product sorterd by dec 
- Same for customer_id in customer file , If found duplicates will pick up the first customer sorted by customer name
- Seen some customer names with special charters eg "Fra9876nk Gasti  ;.,.,neau", Since no insturction given on how to handle this special chareters , For now leaving as is, If we required can remove such special character
- During join to prepare fact_orders for non matching customer and product join , Lookup fields have kept 'UNKNOWN' since not specified what to do with this, IF required i can handle accordigily and filter out this records, 
- For fact_order prepration applied left join and not removed any non joining records
- To read excel source right approch would be to use this "https://medium.com/@amitjoshi7/how-to-read-excel-files-using-pyspark-in-databricks-637bb21b90be" but in databricks free edition we can not install any maven depdency i have used this "https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_excel.html" 


Below are the layers and transformations

Actual data saved in '**Prod**' schema

**RAW layer**
Column names are formatted , below tables created with as is data 
- raw_products 
- raw_orders
- raw_customers

**Enriched layer**
Data types are formatted , Duplicate records are removed, Below are the tables
- enriched_products
- enriched_orders
- enriched_customers

**Fact layer**
3 data set combined to create fact table with required lookup columns from the product and customer table
Profit is rounded to 2 decimal

**Agg layer**
Fact order profit data aggregated by required fields  

In [0]:
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from common_functions import * 

In [0]:

from pyspark.sql import SparkSession 
import pyspark.pandas as ps

products_df = spark.read.csv(
    '/Volumes/workspace/interview/interview_volume/Products.csv',
    header=True,        # Use first line as column names
    quote='"',        # Handle commas inside double quotes
    escape='"',
    inferSchema=True,   # Automatically infer data types
)

orders_df = spark.read.json(
    '/Volumes/workspace/interview/interview_volume/Orders.json',
    multiLine=True
)

customer_psdf = ps.read_excel(
    "/Volumes/workspace/interview/interview_volume/Customer.xlsx",
    dtype=str,
    sheet_name="Worksheet"   # optional
)
customer_df = customer_psdf.to_spark()




/databricks/python/lib/python3.11/site-packages/pyspark/pandas/utils.py:1035: PandasAPIOnSparkAdviceWarning: The config 'spark.sql.ansi.enabled' is set to True. This can cause unexpected behavior from pandas API on Spark since pandas API on Spark follows the behavior of pandas, not SQL.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/databricks/python/lib/python3.11/site-packages/pyspark/pandas/namespace.py:1147: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_excel(
/databricks/python/lib/python3.11/site-packages/pyspark/pandas/namespace.py:1147: FutureWarning: convert_float is deprecated and will be removed in a future version.
  return pd.read_excel(
/databricks/python/lib/python3.11/site-packages/pyspark/pandas/namespace.py:1147: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns

hai


In [0]:
env_name = 'prod'

In [0]:
product_raw_df,order_raw_df,customer_raw_df=create_raw_layer(products_df, orders_df, customer_df)
save_table(spark, product_raw_df, env_name, 'raw_products')
save_table(spark,order_raw_df, env_name, 'raw_orders')
save_table(spark, customer_raw_df, env_name, 'raw_customer')

products_df_raw = read_table(spark, env_name, 'raw_products')
orders_df_raw = read_table(spark, env_name, 'raw_orders')
customer_df_raw = read_table(spark, env_name, 'raw_customer')

products_df_enriched, orders_df_enriched, customer_df_enriched = \
create_enriched_layer(products_df_raw, orders_df_raw, customer_df_raw)

save_table(spark, products_df_enriched, env_name, 'enriched_products')
save_table(spark, orders_df_enriched, env_name, 'enriched_orders')
save_table(spark, customer_df_enriched, env_name, 'enriched_customer')


fact_df = create_fact_table(products_df_enriched, orders_df_enriched, customer_df_enriched)
save_table(spark, fact_df, env_name, 'fact_orders')


agg_df= create_agg_table(fact_df)
save_table(spark, agg_df, env_name, 'agg_orders')


In [0]:
%sql
-- Create an aggregate table that shows profit by year
select order_year, sum(profit)profit
from prod.agg_orders
group by 1
order by 1 desc

order_year,profit
2017,111084.95999999983
2016,65073.229999999996
2015,63073.09
2014,39185.75


In [0]:
%sql
-- Create an aggregate table that shows profit by order_year, product_category
select order_year, product_category, sum(profit)profit
from prod.agg_orders
group by 1, 2
order by 1 desc,2

order_year,product_category,profit
2017,Furniture,3041.540000000001
2017,Office Supplies,44273.12000000001
2017,Technology,63281.78999999989
2017,UNKNOWN,488.5100000000001
2016,Furniture,6889.499999999998
2016,Office Supplies,34555.52999999996
2016,Technology,23223.760000000006
2016,UNKNOWN,404.43999999999994
2015,Furniture,3027.179999999996
2015,Office Supplies,24519.39000000002


In [0]:
%sql
-- Create an aggregate table that shows profit by customer_name
select customer_name, sum(profit)profit
from prod.agg_orders
group by 1
order by 1 desc

customer_name,profit
_12312Patrick Bzostek,228.77
[]-=;''Becky Pak,647.0000000000001
Zuschuss Donatelli,248.75
Zuschuss Carroll,-1032.3599999999997
Yoseph Carroll,1305.6399999999999
Yana Sorensen,1778.31
Xylona Preis,621.3699999999999
William Brown,714.5199999999999
Vivian Mathis,116.64
Vivek Sundaresam,-263.08000000000004


In [0]:
%sql
-- Create an aggregate table that shows profit by order_year, customer_name
select order_year, customer_name, sum(profit)profit
from prod.agg_orders
group by 1,2
order by 1 desc

order_year,customer_name,profit
2017,Doug Jacobs,669.79
2017,Adam Shillingsburg,-331.65
2017,Dianna Wilson,162.99
2017,Eva Jacobs,140.01999999999998
2017,Emily Phan,2.6000000000000085
2017,Aimee Bixby,39.99999999999999
2017,Keith Dawkins,134.83999999999997
2017,Sanjit Engle,-236.82999999999998
2017,Ken Black,240.99
2017,Darrin Martin,106.77000000000001
